In [1]:
import numpy as np
from PIL import Image
import sys
sys.path.append("../")

import caffe
import vis

import time

caffe.set_device(2)
caffe.set_mode_gpu()
# the demo image is "2007_000129" from PASCAL VOC

# load image, switch to BGR, subtract mean, and make dims C x H x W for Caffe
im = Image.open('../demo/image.jpg')
in_ = np.array(im, dtype=np.float32)
in_ = in_[:,:,::-1]
in_ -= np.array((104.00698793,116.66876762,122.67891434))
in_ = in_.transpose((2,0,1))

time_start=time.time()
# load net
net = caffe.Net('deploy.prototxt', 'snapshot/train_iter_100000.caffemodel', caffe.TEST)
# shape for input (data blob is N x C x H x W), set data
net.blobs['data'].reshape(1, *in_.shape)
net.blobs['data'].data[...] = in_
time_end=time.time()
print('load cost',time_end-time_start)
# run net and take argmax for prediction
net.forward()
out = net.blobs['Crop5'].data[0].argmax(axis=0)
time_end2=time.time()
print('segment cost',time_end2-time_end)

# visualize segmentation in PASCAL VOC colors
voc_palette = vis.make_palette(21)
out_im = Image.fromarray(vis.color_seg(out, voc_palette))
out_im.save('../demo/output.png')
masked_im = Image.fromarray(vis.vis_seg(im, out, voc_palette))
masked_im.save('../demo/visualization.jpg')

('load cost', 0.7934238910675049)


KeyError: 'score'